In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib_venn import venn2

# Data

I performed an open ionbot and msfragger search (oxididation of M variable, max 2 missed cleavages).

See SOP document for more details on how MSFragger was ran.

In [2]:
#set this your local folder as the data was pushed to the repo as well
folder = "/home/compomics/tools/ionbot_paper"
#datasets = ["PXD001468","PXD001468_A"]
datasets = ["PXD001250"]

 # Results
 
Load ionbot PSM results (first-matches + coeluting matches + features) + protein results.

In [3]:
def contains_missed_cleavage(x):
    if "K" in x[:-1]:
        return True
    if "R" in x[:-1]:
        return True
    return False

def is_in_s(x):
    return x in s

def get_scan_msfragger(x):
    return int(x.split('.')[-2])
def get_spectrum_file_msfragger(x):
    return x.split('.')[0]

def replace_L_I(x):
    return x.replace("L","I")

def is_decoy(x):
    return "decoy_" in x

In [4]:
results = {}
metrics = []
fi = 1
for i,dataset in enumerate(datasets):
    print(dataset)
    #Read ionbot results
    if "PXD001468" in dataset:
        def get_scan_ionbot(x):
            return int(x.split('.')[1])
        def get_spectrum_file_ionbot(x):
            return x.split('.')[0]
    else:
        def get_scan_ionbot(x):
            print(x)
            return int(x.split('=')[-1])
        def get_spectrum_file_ionbot(x):
            return x.split('.')[0]
    folder2 = folder + "/" + dataset
    ionbot = pd.read_csv("%s/ionbot.first.csv"%folder2)
    ionbot["rank"] = ["first"]*len(ionbot)
    tmp = pd.read_csv("%s/ionbot.lower.csv"%folder2)
    tmp["rank"] = ["lower"]*len(tmp)
    ionbot = pd.concat([ionbot,tmp])
    #NO IDEA WHAT HAPPENED HERE...
    ionbot = ionbot.drop_duplicates("ionbot_match_id")
    tmp = pd.read_csv("%s/ionbot.features.csv"%folder2)
    ionbot = ionbot.merge(tmp,on="ionbot_match_id",how="left")
    ionbot = ionbot[(ionbot["database"]=="T")&(ionbot["q-value"]<=0.01)]
    print(len(ionbot))
    if "PXD001468" in dataset:
        ionbot["scan"] = ionbot["spectrum_title"].apply(get_scan_ionbot)
        ionbot["sfile"] = ionbot["spectrum_title"].apply(get_spectrum_file_ionbot)
    else:
        ionbot["sfile"] = ionbot["spectrum_file"].apply(get_spectrum_file_ionbot)
    #correction, later implemented in ionbot
    ionbot["contains_missed_cleavage"] = ionbot["matched_peptide"].apply(contains_missed_cleavage)    
    ionbot["num_passed"] = ionbot.groupby(["sfile","scan"])["scan"].transform("count")
    ionbot = ionbot[(ionbot["num_passed"]==1) |
            (ionbot["rank"]=="lower") |
            ((ionbot["num_passed"]>1) & (ionbot["rank"]=="first") & (ionbot["unexpected_modification"].isna()) & (ionbot["contains_missed_cleavage"]==False))]
    print(len(ionbot))
    #need to re-assign best_psm
    best_psms = ionbot.sort_values("psm_score").drop_duplicates(["sfile","scan"], keep="last")
    best_psm_ids = set(best_psms["ionbot_match_id"])
    ionbot["rank"] = ionbot["ionbot_match_id"].apply(
    lambda x: "first" if x in best_psm_ids else "lower")    
    ionbot["PSM"] = ionbot["scan"].astype(str) + ionbot["sfile"] + ionbot["database_peptide"].apply(replace_L_I)
    print(len(ionbot))
    results["ionbot_"+dataset] = ionbot

    #Read MSfragger (Prophet) result
    fragger = pd.read_csv("%s/msfragger/%s/noprotein/psm.tsv"%(folder,dataset),sep="\t")
    fragger["scan"] = fragger["Spectrum"].apply(get_scan_msfragger)
    #sfile is used to filter on spectrum file
    fragger["sfile"] = fragger["Spectrum"].apply(get_spectrum_file_msfragger)
    fragger["PSM"] = fragger["scan"].astype(str) + fragger["sfile"] + fragger["Peptide"].apply(replace_L_I)

    results["fragger_"+dataset] = fragger

    continue
    #adding MS2PIP correlation and retention time prediction error features:
    data_folder = "/home/compomics/extra_disk/data/%s/"%dataset
    sfiles = fragger["sfile"].unique()
    X = []
    for sf in sfiles:
        tmp = pd.read_csv("%s/%s_fragger.peprec_ionbotpeprec/ionbot.csv"%(data_folder,sf))
        tmp["sfile"] = [sf]*len(tmp)
        X.append(tmp)
    fragger_stats = pd.concat(X)
    if dataset == "PXD001468":
        def get_scan(x):
            return int(x.split(".")[1])
        fragger_stats['scan'] = fragger_stats["title"].apply(get_scan)
    tmp = fragger_stats[["scan","sfile","corr","rt_diff_A"]]
    fragger = fragger.merge(tmp,on=["scan","sfile"],how="left")

    #Read MSfragger (Percolator) results
    #read all tsv files into DataFrame
    datasets2 = ionbot["sfile"].unique()
    X = []
    for ds in datasets2:
        d = pd.read_csv("%s/msfragger/%s/%s.tsv"%(folder,dataset,ds),sep="\t")
        d["sfile"] = [ds]*len(d)
        X.append(d)
    fragger_perc = pd.concat(X)
    fragger_perc.rename(columns={'scannum':'scan'}, inplace=True)
    fragger_perc["is_decoy"] = fragger_perc["protein"].apply(is_decoy)    
    #add statistics
    X = []
    with open("%s/msfragger/%s/o"%(folder,dataset)) as f:
        f.readline()
        for row in f:
            l = row.rstrip().split('\t')
            sfile = l[0].split('.')[0]
            scan = int(l[0].split('.')[-2])
            X.append([scan,sfile,float(l[2]),float(l[3])])
    stats = pd.DataFrame(X,columns=["scan","sfile","q-value","PEP"])
    fragger_perc = fragger_perc.merge(stats,on=["scan","sfile"],how="left")
    fragger_perc["q-value"].fillna(1,inplace=True)
    fragger_perc["PEP"].fillna(1,inplace=True)
    fragger_perc = fragger_perc[(fragger_perc["is_decoy"]==False)&(fragger_perc["q-value"]<=0.01)]
    fragger_perc["PSM"] = fragger_perc["scan"].astype(str) + fragger_perc["sfile"] + fragger_perc["peptide"].apply(replace_L_I)
    fragger_stats = fragger[["scan","sfile","corr","rt_diff_A"]]
    fragger_perc = fragger_perc.merge(fragger_stats,on=["scan","sfile"],how="left")





    

PXD001250
433357
428565
428565


In [5]:
for d in results:
    print(d)
    print(len(results[d]))

ionbot_PXD001250
428565
fragger_PXD001250
444478


In [6]:
to_write = pd.concat(metrics)
sns.boxplot(x="dataset",y="value",hue="type",data=to_write)

ValueError: No objects to concatenate

In [ ]:
to_write[to_write["dataset"]=="TMTCPTAC"]["type"].value_counts()

M    32364
I    25811
Name: type, dtype: int64